# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,38.86,70,78,32.77,TF,1706635768
1,1,vorgashor,67.5833,63.9500,7.41,95,100,12.24,RU,1706635769
2,2,jamestown,42.0970,-79.2353,36.99,90,100,11.50,US,1706635557
3,3,constantia,44.1833,28.6500,29.08,71,83,13.35,RO,1706635513
4,4,yellowknife,62.4560,-114.3525,19.92,100,100,15.01,CA,1706635770


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
hover_columns = ['City', 'Humidity']
plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size=city_data_df['Humidity'], 
                                  width=800, height=550, c='Humidity', cmap='spectral', alpha=0.5, hover_cols=hover_columns)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 70) & (city_data_df['Max Temp'] > 65) & (city_data_df['Humidity'] < 80)]

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df

C:\Users\K\AppData\Local\Temp\ipykernel_10992\357253380.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,i-n-salah,27.1935,2.4607,68.32,23,0,18.70,DZ,1706635802
98,98,minbu,20.1833,94.8833,65.62,38,61,3.42,MM,1706635833
116,116,leeton,-34.5667,146.4000,68.09,38,0,6.96,AU,1706635845
117,117,lebu,-37.6167,-73.6500,65.44,63,0,14.54,CL,1706635846
130,130,tippi,7.2000,35.4500,69.87,76,88,3.29,ET,1706635854
168,168,broken hill,-31.9500,141.4333,66.22,40,0,21.56,AU,1706635878
200,200,tazacorte,28.6290,-17.9293,68.97,73,0,3.44,ES,1706635900
217,217,quellon,-43.1167,-73.6167,65.50,59,10,7.14,CL,1706635910
234,234,san patricio,28.0170,-97.5169,65.91,54,100,1.70,US,1706635741
300,300,praia da vitoria,38.7333,-27.0667,65.75,77,40,10.69,PT,1706635834


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
59,i-n-salah,DZ,27.1935,2.4607,23,
98,minbu,MM,20.1833,94.8833,38,
116,leeton,AU,-34.5667,146.4000,38,
117,lebu,CL,-37.6167,-73.6500,63,
130,tippi,ET,7.2000,35.4500,76,
168,broken hill,AU,-31.9500,141.4333,40,
200,tazacorte,ES,28.6290,-17.9293,73,
217,quellon,CL,-43.1167,-73.6167,59,
234,san patricio,US,28.0170,-97.5169,54,
300,praia da vitoria,PT,38.7333,-27.0667,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 1,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_response.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
i-n-salah - nearest hotel: باجودة
minbu - nearest hotel: Rolex Guesthouse
leeton - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
tippi - nearest hotel: Tepi genet penision
broken hill - nearest hotel: Ibis Styles
tazacorte - nearest hotel: App Leyma
quellon - nearest hotel: Aliwen
san patricio - nearest hotel: No hotel found
praia da vitoria - nearest hotel: Salles
hays - nearest hotel: Fall Creek Inn
garachico - nearest hotel: Residencia El Drago
san julian - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
lampazos de naranjo - nearest hotel: Hotel Lampazos
beacon square - nearest hotel: Fairfield Inn & Suites by Marriott Holiday Tarpon Springs
nejo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
59,i-n-salah,DZ,27.1935,2.4607,23,باجودة
98,minbu,MM,20.1833,94.8833,38,Rolex Guesthouse
116,leeton,AU,-34.5667,146.4000,38,No hotel found
117,lebu,CL,-37.6167,-73.6500,63,No hotel found
130,tippi,ET,7.2000,35.4500,76,Tepi genet penision
168,broken hill,AU,-31.9500,141.4333,40,Ibis Styles
200,tazacorte,ES,28.6290,-17.9293,73,App Leyma
217,quellon,CL,-43.1167,-73.6167,59,Aliwen
234,san patricio,US,28.0170,-97.5169,54,No hotel found
300,praia da vitoria,PT,38.7333,-27.0667,77,Salles


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_columns = ['City', 'Humidity', 'Hotel Name', 'Country']
hotel_plot = plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size=city_data_df['Humidity'], 
                                  width=800, height=550, c='Humidity', cmap='spectral', alpha=0.5, hover_cols=hotel_columns)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City,Hotel Name,Country)